In [1]:
import logging
import pickle
import numpy as np
from openmmtools.integrators import PeriodicNonequilibriumIntegrator
from simtk import unit
from simtk import openmm
import argparse
import os
import time
import mdtraj as md
from perses.rjmc.geometry import FFAllAngleGeometryEngine

In [2]:
# Set up logger
_logger = logging.getLogger()
_logger.setLevel(logging.INFO)


In [3]:
# Define lambda functions
x = 'lambda'
DEFAULT_ALCHEMICAL_FUNCTIONS = {
                             'lambda_sterics_core': x,
                             'lambda_electrostatics_core': x,
                             'lambda_sterics_insert': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_sterics_delete': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_insert': f"select(step({x} - 0.5), 2.0 * ({x} - 0.5), 0.0)",
                             'lambda_electrostatics_delete': f"select(step({x} - 0.5), 1.0, 2.0 * {x})",
                             'lambda_bonds': x,
                             'lambda_angles': x,
                             'lambda_torsions': x}

In [4]:
# Define simulation parameters
nsteps_eq = 10
nsteps_neq = 40000 # 80 ps
neq_splitting='V R H O R V'
timestep = 2.0 * unit.femtosecond
platform_name = 'CUDA'

In [5]:
# Read in htf
outdir = "/data/chodera/zhangi/perses_benchmark/neq/7/13/"
i = os.path.basename(os.path.dirname(outdir))
phase = 'vacuum'
with open(os.path.join(outdir, f"{i}_{phase}.pickle"), 'rb') as f:
    htf = pickle.load(f)
system = htf.hybrid_system


In [7]:
# Read in cache
old_name = 'thr'
with open(os.path.join(outdir, f"blues/{old_name}_pos_hybrid.npy"), 'rb') as f:
    pos_hybrid = np.load(f)

In [17]:
# Get equilbrium snapshot
sim_number = 0
old_positions = pos_hybrid[int(sim_number)] * unit.nanometer


In [18]:
# Make geometry engine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                                use_sterics=False,
                                                n_bond_divisions=100,
                                                n_angle_divisions=180,
                                                n_torsion_divisions=360,
                                                verbose=True,
                                                storage=None,
                                                bond_softening_constant=1.0,
                                                angle_softening_constant=1.0,
                                                neglect_angles = False,
                                                use_14_nonbondeds = True)


In [19]:
# Compute new positions
from openmmtools.constants import kB
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
new_positions, logp_proposal = geometry_engine.propose(htf._topology_proposal, old_positions, beta, validate_energy_bookkeeping=True)


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
RDKit WARNING: [21:28:03] Enabling RDKit 2019.09.3 jupyter extensions
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 2
INFO:geometry:Atom index proposal order is [15, 14]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 21 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 36 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 42 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing ref

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 2 new atoms
INFO:geometry:	reduced angle potential = 0.5184420744637405.
INFO:geometry:	reduced angle potential = 0.007898630335858461.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 21 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 42.376594799837925


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.3462552262411282), ('CustomAngleForce', 1.2789640293281703), ('CustomTorsionForce', 0.180013587198817), ('CustomBondForce', 6.860867910208524)]


INFO:geometry:total reduced energy added from growth system: 8.66610075297664
INFO:geometry:final reduced energy 51.0426956404324
INFO:geometry:sum of energies: 51.042695552814564
INFO:geometry:magnitude of difference in the energies: 8.761783476529672e-08
INFO:geometry:Final logp_proposal: 20.370300630284


In [21]:
# Set positions and generate hybrid 
htf._old_positions = old_positions
htf._new_positions = new_positions
htf._hybrid_positions = htf._compute_hybrid_positions()
positions = htf.hybrid_positions


In [23]:
# Set up integrator
integrator = PeriodicNonequilibriumIntegrator(DEFAULT_ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep)


In [24]:
# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*system.getDefaultPeriodicBoxVectors())
context.setPositions(positions)


In [25]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)


In [26]:
# Run eq forward (0 -> 1)
initial_time = time.time()
integrator.step(nsteps_eq)
elapsed_time = (time.time() - initial_time) * unit.seconds
_logger.info(f'Equilibrating at lambda = 0, took: {elapsed_time / unit.seconds} seconds')

# Run neq forward (0 -> 1)
forward_works_master = list()
forward_neq_old, forward_neq_new = list(), list()
forward_works = [integrator.get_protocol_work(dimensionless=True)]
for fwd_step in range(nsteps_neq):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    forward_works.append(integrator.get_protocol_work(dimensionless=True))
    if fwd_step % 750 == 0:
        _logger.info(f'forward NEQ step: {fwd_step}, took: {elapsed_time / unit.seconds} seconds')
        pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
        old_pos = np.asarray(htf.old_positions(pos))
        new_pos = np.asarray(htf.new_positions(pos))
        forward_neq_old.append(old_pos)
        forward_neq_new.append(new_pos)
forward_works_master.append(forward_works)

# Read in ala cache
with open(os.path.join(args.dir, f"{args.new_name}_pos_hybrid.npy"), 'rb') as f:
    ala_pos_hybrid = np.load(f)

# Read in indices of uncorrelated ala snapshots
with open(os.path.join(args.dir, f"{args.new_name}_indices.npy"), 'rb') as f:
    ala_indices = np.load(f)

# Get equilbrium snapshot of ala
positions = ala_pos_hybrid[ala_indices[int(args.sim_number)]]
context.setPositions(positions)

# Run eq reverse (1 -> 0)
initial_time = time.time()
integrator.step(nsteps_eq)
elapsed_time = (time.time() - initial_time) * unit.seconds
_logger.info(f'Equilibrating at lambda = 1, took: {elapsed_time / unit.seconds} seconds')

# Run neq reverse (1 -> 0)
reverse_works_master = list()
reverse_neq_old, reverse_neq_new = list(), list()
reverse_works = [integrator.get_protocol_work(dimensionless=True)]
for rev_step in range(nsteps_neq):
    initial_time = time.time()
    integrator.step(1)
    elapsed_time = (time.time() - initial_time) * unit.seconds
    reverse_works.append(integrator.get_protocol_work(dimensionless=True))
    if rev_step % 750 == 0:
        _logger.info(f'reverse NEQ step: {rev_step}, took: {elapsed_time / unit.seconds} seconds')
        pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
        old_pos = np.asarray(htf.old_positions(pos))
        new_pos = np.asarray(htf.new_positions(pos))
        reverse_neq_old.append(old_pos)
        reverse_neq_new.append(new_pos)
reverse_works_master.append(reverse_works)


INFO:root:Equilibrating at lambda = 0, took: 1.204496145248413 seconds
INFO:root:forward NEQ step: 0, took: 0.004120349884033203 seconds
INFO:root:forward NEQ step: 750, took: 0.002613544464111328 seconds
INFO:root:forward NEQ step: 1500, took: 0.0030274391174316406 seconds
INFO:root:forward NEQ step: 2250, took: 0.0026171207427978516 seconds
INFO:root:forward NEQ step: 3000, took: 0.0004639625549316406 seconds
INFO:root:forward NEQ step: 3750, took: 0.0004825592041015625 seconds
INFO:root:forward NEQ step: 4500, took: 0.01919102668762207 seconds
INFO:root:forward NEQ step: 5250, took: 0.00044798851013183594 seconds
INFO:root:forward NEQ step: 6000, took: 0.0005881786346435547 seconds
INFO:root:forward NEQ step: 6750, took: 0.0004055500030517578 seconds
INFO:root:forward NEQ step: 7500, took: 0.0004830360412597656 seconds
INFO:root:forward NEQ step: 8250, took: 0.0038754940032958984 seconds
INFO:root:forward NEQ step: 9000, took: 0.00043463706970214844 seconds
INFO:root:forward NEQ ste

KeyboardInterrupt: 

In [ ]:
# Save works
i = 13
with open(os.path.join(args.dir, f"{i}_{phase}_{sim_number}_forward.npy"), 'wb') as f:
    np.save(f, forward_works_master)
with open(os.path.join(args.dir, f"{i}_{phase}_{sim_number}_reverse.npy"), 'wb') as f:
    np.save(f, reverse_works_master)
